In [288]:
from delphi_epidata import Epidata
import pandas as pd

In [289]:
# Data has to be acquired in batches since there is a length limit in the API. Acquiring in batches of 3 days

def retrieveData(month,start,end,df):
    if month < 10:
        month = f'0{month}'
    else:
        month = str(month)
    if start < 10:
        start = f'0{start}'
    else:
        start = str(start)
    if end < 10:
        end = f'0{end}'
    else:
        end = str(end)
        
    for i in range(int(start),int(end)+1,3):
        if i+1 < 10:
            if i == int(start):
                startdate = f'0{i}'
            else:
                startdate = f'0{i+1}'
        else:
            if i == int(start):
                startdate = i
            else:
                startdate = i+1
        enddate = i+3
        if i + 3 < 10:
            enddate = f'0{enddate}'
        
        if int(enddate) > int(end):
            enddate = int(end)
            
        startdate = str(startdate)
        enddate = str(enddate)
        
        if startdate == enddate:
            break
        
        res = Epidata.covidcast('fv-survey', 'raw_wcli', 'day', 'county', [Epidata.range(int(f'2020{month}{startdate}'), int(f'2020{month}{enddate}'))], geo_value='*')
        if res['result'] == 1:
            df = df.append(res['epidata'])
            print(f'retreival for {startdate} to {enddate} succeeded')
        else:
            print(f'Retreival for {startdate} to {enddate} failed')
    
    return df
        

In [ ]:
df = retrieveData(4,8,30, pd.DataFrame())
df = retrieveData(5,1,27, df)

In [294]:
counties = []
for county in df['geo_value']:
    counties.append(county)
counties = list(set(counties))
cleanedDf = pd.DataFrame(dtype=str)

In [295]:
for county in counties:
    countyData = df[df['geo_value'] == county]
    countyData = countyData.reset_index()
    countyData = countyData.drop(['index'], axis=1)
    
    # Time series is in rows unlike applemobility data, converting to columns instead
    temp = pd.DataFrame(columns=['geo_value', *countyData['time_value']])
    
    # Renaming columns to match applemobility data
    columns = ["geo_value"]
    for i,column in enumerate(temp.columns):
        if i == 0:
            continue
        column = list(str(column))
        column.insert(4, '-')
        column.insert(7, '-')
        columns.append(''.join(column))
    temp.columns = columns
    
    values = []
    for i,column in enumerate(columns):
        if i == 0:
            values.append(str(countyData['geo_value'].values[0]))
            continue
        else:
            columnAdj = int(column.replace('-', ''))
            value = countyData[countyData['time_value'] == columnAdj].values[0][3]
            values.append(value)
    
    temp = temp.append(pd.Series(values, index=temp.columns), ignore_index=True)
    cleanedDf = cleanedDf.append(temp, ignore_index=True)

In [296]:
cleanedDf = cleanedDf.reindex(sorted(cleanedDf.columns), axis=1)
cleanedDf = cleanedDf.fillna(0)
firstCol = cleanedDf.pop('geo_value')
cleanedDf.insert(0, 'geo_value', firstCol)

In [298]:
cleanedDf.to_csv('covidsymptoms_fb.csv')